In [1]:
import local_models.local_models
import local_models.algorithms
import local_models.utils
import local_models.linear_projections
import local_models.loggin
import local_models.TLS_models
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.cluster
from importlib import reload
from ml_battery.utils import cmap
import matplotlib as mpl
import sklearn.datasets
import sklearn.decomposition
import logging
import ml_battery.log
import time
import os
import mayavi
import mayavi.mlab
import string
import subprocess
import functools
import cv2
import itertools

#on headless systems, tmux: "Xvfb :1 -screen 0 1280x1024x24 -auth localhost", then "export DISPLAY=:1" in the jupyter tmux
mayavi.mlab.options.offscreen = True



logger = logging.getLogger(__name__)

#reload(local_models.local_models)
#reload(lm)
#reload(local_models.loggin)
#reload(local_models.TLS_models)
np.warnings.filterwarnings('ignore')


/usr/local/lib/python3.5/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [424]:
import importlib; importlib.reload(quadrics_utils)

<module 'quadrics_utils' from '/home/brown/python_packages/local_models/examples/quadrics_utils.py'>

In [1860]:
def collect_best(expr, measure=sympy.count_ops):
    best = expr
    best_score = measure(expr)
    perms = itertools.permutations(expr.free_symbols)
    permlen = np.math.factorial(len(expr.free_symbols))
    print(permlen)
    for i, perm in enumerate(perms):
        if (permlen > 1000) and not (i%int(permlen/100)):
            print(i)
        collected = sympy.collect(expr, perm)
        if measure(collected) < best_score:
            best_score = measure(collected)
            best = collected
        else:
            factored = sympy.factor(expr)
            if measure(factored) < best_score:
                best_score = measure(factored)
                best = factored
    return best
    
def product(args):
    arg = next(args)
    try:
        return arg*product(args)
    except:
        return arg
    
def rcollect_best(expr, measure=sympy.count_ops):
    best = collect_best(expr, measure)
    best_score = measure(best)
    if expr == best:
        return best
    if isinstance(best, sympy.Mul):
        return product(map(rcollect_best, best.args))
    if isinstance(best, sympy.Add):
        return sum(map(rcollect_best, best.args))

In [3579]:
def derive_quadratic_orthogonal_projection_1D_polynomial(n):
    import sympy
    
    Q_sym = sympy.symarray("q", (n+1, n+1))
    Q = sympy.Matrix(np.zeros((n+1,n+1), dtype=int))
    for i, j in itertools.product(range(n+1), range(n+1)):
        if i == n or j == n or i == j:
            Q[i,j] = Q_sym[max(i,j),min(i,j)]
    print(Q)

    x_sym = sympy.symarray("x", n+1)
    X = sympy.Matrix(np.ones((n+1, 1), dtype=int))
    for i in range(n):
        X[i] = x_sym[i]
        
    P = sympy.Matrix(np.zeros((n-1, n+1), dtype=int))
    for i in range(n-1):
        P[i,0] = X[i+1]
        P[i,i+1] = -X[0]
    
    QXP = P*Q*X
    
    other_dims_as_x0 = [sympy.solve(QXP[i], X[i+1])[0] for i in range(n-1)] 
    
    XQX = sympy.expand((X.T*Q*X)[0])
    XQX_as_x0 = XQX.subs({X[i+1]:other_dims_as_x0[i] for i in range(n-1)})
    for sub in other_dims_as_x0:
        XQX_as_x0 *= sympy.fraction(sub)[1]**2
    XQX_as_x0 = sympy.cancel(XQX_as_x0)
    XQX_as_x0 = sympy.simplify(XQX_as_x0)
    XQX_as_x0 = sympy.poly(XQX_as_x0, X[0])
    
    return (X, Q, XQX_as_x0, other_dims_as_x0)
    

In [ ]:
def collectify_polynomial_coefficients(poly):
    return [rcollect_best(formula) for formula in poly.all_coeffs()]

In [3011]:
sorted(map(lambda x: x**2, range(4)))

[0, 1, 4, 9]

In [ ]:
def funcify(signature, returnable):
    return "def {}: return {}\n".format(signature, returnable)

def librarify_quadric_equations():
    import sympy
    from sympy.abc import a,b,c,d,e,f,g,x,y,z

    Q = a*x**2 + b*y**2 + c*z**2 + 2*e*x + 2*f*y + 2*g*z + d
    y_as_x_num = f*x
    y_as_x_den = e-(b-a)*x
    y_as_x = y_as_x_num/y_as_x_den
    z_as_x_num = g*x
    z_as_x_den = e-(c-a)*x
    z_as_x = z_as_x_num/z_as_x_den
    Q_as_x = Q.subs({
        y: y_as_x,
        z: z_as_x,
    })

    bigQ = sympy.expand(sympy.simplify(Q_as_x*y_as_x_den**2*z_as_x_den**2))

    coeffs = list(map(sympy.factor, sympy.poly(bigQ,x).all_coeffs()))

    collected = []
    for coeff in coeffs:
        collected.append(rcollect_best(coeff))

    k_mat = sympy.Matrix(collected)
    k_jac = k_mat.jacobian([a,b,c,d,e,f,g])

    with open("quadrics_utils.py", "w") as f:
        Q_func = funcify("Q(a,b,c,d,e,f,g,x,y,z)",str(Q))
        y_as_x_func = funcify("y_as_x(a,b,c,d,e,f,g,x)", str(y_as_x))
        z_as_x_func = funcify("z_as_x(a,b,c,d,e,f,g,x)", str(z_as_x))
        Q_as_x_func = funcify("Q_as_x(a,b,c,d,e,f,g,x)", str(Q_as_x))
        k_mat_func = funcify("k_mat(a,b,c,d,e,f,g)", str(k_mat.transpose().tolist()[0]))
        k_jac_func = funcify("k_jac(a,b,c,d,e,f,g)", str(k_jac.tolist()))
        individual_k_funcs = [funcify("k{:01d}(a,b,c,d,e,f,g)".format(i), str(eq)) for i,eq in enumerate(collected[::-1])]
        list(map(f.write, [Q_func, y_as_x_func, z_as_x_func, Q_as_x_func, k_mat_func, k_jac_func] + individual_k_funcs))